## Alternative Models

Now that we have explored Multiple Linear Regression as well as Lasso, let's explore different follow-up methods to address the issues we noticed before.

In [3]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import pickle
from collections import Counter
import sklearn as sk
from sklearn.preprocessing import Imputer
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor as rf
%matplotlib inline

First, let's load in the data and clean it as we did earlier.

In [4]:
# Load cleaned NCD data from pickled files
out = open('data/clean/deaths_100k.p', 'r')
deaths_100k = pickle.load(out)
out.close()
out = open('data/clean/risk.p', 'r')
risk_of_death = pickle.load(out)
out.close()
out = open('data/clean/crops.p', 'r')
crops = pickle.load(out)
out.close()
out = open('data/clean/meat.p', 'r')
meat = pickle.load(out)
out.close()
out = open('data/clean/var_desc_livestock.p', 'r')
livestock_desc = pickle.load(out)
out.close()
out = open('data/clean/var_desc_crops.p', 'r')
crops_desc = pickle.load(out)
out.close()

In [5]:
# Get response variables
deaths_100k_all_2000 = deaths_100k['all'][2000]
deaths_100k_cancer_2000 = deaths_100k['cancer'][2000]
deaths_100k_cardio_2000 = deaths_100k['cardio'][2000]
deaths_100k_diabetes_2000 = deaths_100k['diabetes'][2000]
deaths_100k_resp_2000 = deaths_100k['resp'][2000]

risk_of_death_2000 = risk_of_death[2000]

In [6]:
# TODO: Turn this process into a function later?
time_period = range(1970, 2000)

# Calculate the mean for each crop/meat over the period 1970-2000
food_1970_2000 = pd.DataFrame(index=risk_of_death.index)

for crop in crops.iterkeys():
    food_1970_2000[crop] = crops[crop][time_period].mean(axis=1)
    
for m in meat.iterkeys():
    food_1970_2000[m] = meat[m][time_period].mean(axis=1)

food_1970_2000.head()

,Ricebran Oil,Oilcrops,Plantains,"Sugar, Raw Equivalent","Beverages, Alcoholic",Roots & Tuber Dry Equiv,Vegetable Oils,Olives (including preserved),Cloves,Millet and products,...,Offals,Bovine Meat,"Molluscs, Other","Fish, Body Oil","Aquatic Animals, Others",Animal fats,Honey,"Offals, Edible",Demersal Fish,Cream
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,2.629000,NaN,14.402000,0.002333,8.539333,7.995000,0.065333,NaN,5.360333,...,9.166000,16.762667,NaN,NaN,NaN,6.513000,0.687667,9.166000,NaN,0.000000
Albania,NaN,7.044333,NaN,50.132333,3.201000,10.253333,20.501000,7.017667,NaN,NaN,...,6.729333,18.681000,0.416000,0.0,0.0,8.995000,0.488667,6.734667,0.604000,0.005333
Algeria,NaN,1.646000,0.0,69.079333,0.108667,15.592333,33.352667,1.432667,0.004667,NaN,...,3.079333,9.742333,0.001000,0.0,0.0,5.180333,0.247333,3.079333,0.950333,NaN
Angola,NaN,3.671667,NaN,26.933333,1.855000,157.302000,21.056000,NaN,0.000000,16.849333,...,3.521333,21.081000,0.000333,0.0,0.0,1.588000,5.557000,3.521333,3.859667,NaN
Argentina,NaN,1.813000,NaN,113.051667,10.766000,41.883333,35.942667,1.235333,0.000333,NaN,...,19.928000,192.834333,0.675000,0.0,0.0,10.627667,0.505000,19.933333,13.161333,0.157000


In [7]:
# list of countries to drop due to being less than 50% full
countries_to_drop = []

for index, row in food_1970_2000.iterrows():
    if row.isnull().sum() > len(row) / 2:
        countries_to_drop.append(index)

print countries_to_drop
print "Number of countries to drop:", len(countries_to_drop)

['Bahrain', 'Belgium', 'Bhutan', 'Burundi', 'Comoros', 'Democratic Republic of the Congo', 'Equatorial Guinea', 'Eritrea', 'Libya', 'Montenegro', 'Papua New Guinea', 'Qatar', 'Serbia', 'Singapore', 'Somalia', 'South Sudan', 'Sudan', 'Syrian Arab Republic']
Number of countries to drop: 18


In [8]:
# Drop the identified countries with very sparse data
food_1970_2000_cleaned = food_1970_2000.drop(countries_to_drop)

In [9]:
cols_to_drop = []

# Identify sparse columns to drop
for col in food_1970_2000_cleaned.columns:
    if food_1970_2000_cleaned[col].isnull().sum() > len(food_1970_2000_cleaned[col]) / 2:
        cols_to_drop.append(col)
        
print cols_to_drop
print "Number of columns to drop:", len(cols_to_drop)

['Ricebran Oil', 'Millet and products', 'Sugar non-centrifugal', 'Molasses', 'Sugar beet', 'Sorghum and products', 'Sunflower seed', 'Sugar Crops', 'Sugar cane', 'Yams', 'Meat, Aquatic Mammals', 'Meat Meal', 'Whey']
Number of columns to drop: 13


In [10]:
# Drop identified crops with very sparse data
food_1970_2000_cleaned = food_1970_2000_cleaned.drop(cols_to_drop, axis=1)

In [11]:
# Impute by mean for each column (i.e. global average per crop)
imp = Imputer(axis=1)
food_1970_2000_cleaned = pd.DataFrame(imp.fit_transform(food_1970_2000_cleaned), index=food_1970_2000_cleaned.index, columns=food_1970_2000_cleaned.columns)

In [12]:
livestock_desc['Description'] = livestock_desc['Description'].astype('string')
crops_desc['Description'] = crops_desc['Description'].astype('string')

In [13]:
# columns to drop because of overlapping subcategories
more_columns_to_drop = ['Groundnuts (in Shell Eq)',
                        'Sweeteners, Other',
                        'Sugar, Refined Equiv',
                        'Rice (Paddy Equivalent)',
                       ]

In [14]:
# drop the columns
food_1970_2000_cleaned = food_1970_2000_cleaned.drop(more_columns_to_drop, axis = 1)

In [15]:
empty_descs = list(livestock_desc[livestock_desc["Description"] == 'nan']['Item'])
empty_descs += list(crops_desc[crops_desc["Description"] == 'nan']['Item'])

In [16]:
# drop the columns
for col in empty_descs:
    if col in food_1970_2000_cleaned.columns:
        food_1970_2000_cleaned = food_1970_2000_cleaned.drop(col, axis = 1)

In [35]:
# drop countries from response variables
deaths_100k_all_2000 = deaths_100k_all_2000.drop(countries_to_drop)
deaths_100k_cancer_2000 = deaths_100k_cancer_2000.drop(countries_to_drop)
deaths_100k_cardio_2000 = deaths_100k_cardio_2000.drop(countries_to_drop)
deaths_100k_diabetes_2000 = deaths_100k_diabetes_2000.drop(countries_to_drop)
deaths_100k_resp_2000 = deaths_100k_resp_2000.drop(countries_to_drop)

risk_of_death_2000 = risk_of_death_2000.drop(countries_to_drop)

Now let's try performing a log transformation on the response variables and see if that addresses the absence of normality we noticed in our earlier models. Note that we only perform this transformation for the age-standardized mortality rates because risk of death is calculated as a percentage already, so there would be no need to transform that data.

In [61]:
deaths_100k_all_2000_log = np.log(deaths_100k_all_2000)
deaths_100k_cancer_2000_log = np.log(deaths_100k_cancer_2000)
deaths_100k_cardio_2000_log = np.log(deaths_100k_cardio_2000)
deaths_100k_diabetes_2000_log = np.log(deaths_100k_diabetes_2000)
deaths_100k_resp_2000_log = np.log(deaths_100k_resp_2000)

Once again we use GridSearchCV to select the best hyperparameters for the Lasso models for each response variable.

In [56]:
lasso = linear_model.Lasso(alpha = 1.0)
lasso.fit(food_1970_2000_cleaned, deaths_100k_all_2000_log)
y_deaths_100k_all_lasso = deaths_100k_all_2000_log.sort_index().loc[food_1970_2000_cleaned.index].values
lasso.score(food_1970_2000_cleaned, y_deaths_100k_all_lasso)
params = {
    'alpha': [0.3,0.6,1.0,1.3,1.6,2.0,2.3,2.6,3.0,4.0,5.0],
    'fit_intercept': [True, False],
    'normalize': [True, False],
}

CV_model = sk.grid_search.GridSearchCV(lasso, param_grid=params, cv=5)
CV_model.fit(food_1970_2000_cleaned, y_deaths_100k_all_lasso)
CV_model.best_params_

{'alpha': 1.0, 'fit_intercept': True, 'normalize': False}

With the best parameters identified, we now fit the Lasso model accordingly.

In [60]:
deaths_all_model_lasso = sm.OLS(y_deaths_100k_all_lasso, food_1970_2000_cleaned)
deaths_all_results_lasso = deaths_all_model_lasso.fit_regularized(alpha = 1.0)
deaths_all_results_lasso.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     41.00
Date:                Tue, 13 Dec 2016   Prob (F-statistic):           1.03e-39
Time:                        11:32:54   Log-Likelihood:                -207.36
No. Observations:                 154   AIC:                             578.7
Df Residuals:                      72   BIC:                             827.8
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------------
Plantains                           0.0035      0.002      1.668      0.100        -0.001     0.008
Sugar, Raw Equivalent               0.0119      0.012      0.998      0.322        -0.012     0.036
Beverages, Alcoholic                     0          0        nan        nan             0         0
Olives (including preserved)             0          0        nan        nan             0         0
Cloves                                   0          0        nan        nan             0         0
Coconuts - Incl Copra                    0          0        nan        nan             0         0
Vegetables, Other                        0          0        nan        nan             0         0
Sesame seed                        -0.0024      0.008     -0.299      0.766        -0.019     0.014
Wine                                     0          0        nan        nan             0         0
Apples and products                      0          0        nan        nan             0         0
Rape and Mustard Oil               -0.0028      0.008     -0.332      0.741        -0.020     0.014
Maize and products                  0.0099      0.001      6.854      0.000         0.007     0.013
Groundnuts (Shelled Eq)                  0          0        nan        nan             0         0
Barley and products                 0.0014      0.006      0.232      0.817        -0.011     0.014
Maize Germ Oil                           0          0        nan        nan             0         0
Beer                                0.0006      0.003      0.221      0.825        -0.005     0.006
Groundnut Oil                            0          0        nan        nan             0         0
Pineapples and products                  0          0        nan        nan             0         0
Pulses, Other and products          0.0385      0.012      3.124      0.003         0.014     0.063
Sugar (Raw Equivalent)              0.0012      0.013      0.090      0.929        -0.025     0.027
Palm Oil                            0.0029      0.009      0.319      0.751        -0.015     0.021
Oilcrops, Other                     0.0002      0.008      0.022      0.982        -0.015     0.016
Dates                               0.0061      0.006      0.985      0.328        -0.006     0.018
Oats                                     0          0        nan        nan             0         0
Soyabeans                                0          0        nan        nan             0         0
Beans                               0.0037      0.011      0.336      0.738        -0.018     0.026
Sesameseed Oil                      0.0070      0.009      0.804      0.424        -0.010     0.024
Grapes and products (excl wine)          0          0        nan        nan        

Even after the logarithmic transformation on the age-standardized mortality rate from all non-communicable diseases, we still reach the same conclusion from the Omnibus and Jarque-Bera tests that the residuals are not normally distributed.

In [62]:
lasso = linear_model.Lasso(alpha = 1.0)
lasso.fit(food_1970_2000_cleaned, deaths_100k_cancer_2000_log)
y_deaths_100k_cancer_lasso = deaths_100k_cancer_2000_log.sort_index().loc[food_1970_2000_cleaned.index].values
lasso.score(food_1970_2000_cleaned, y_deaths_100k_cancer_lasso)
params = {
    'alpha': [0.3,0.6,1.0,1.3,1.6,2.0,2.3,2.6,3.0,4.0,5.0],
    'fit_intercept': [True, False],
    'normalize': [True, False],
}

CV_model = sk.grid_search.GridSearchCV(lasso, param_grid=params, cv=5)
CV_model.fit(food_1970_2000_cleaned, y_deaths_100k_cancer_lasso)
CV_model.best_params_

{'alpha': 2.6, 'fit_intercept': True, 'normalize': False}

In [64]:
deaths_cancer_model_lasso = sm.OLS(y_deaths_100k_cancer_lasso, food_1970_2000_cleaned)
deaths_cancer_results_lasso = deaths_cancer_model_lasso.fit_regularized(alpha = 2.6)
deaths_cancer_results_lasso.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.947
Method:                 Least Squares   F-statistic:                     34.28
Date:                Tue, 13 Dec 2016   Prob (F-statistic):           4.76e-37
Time:                        11:42:01   Log-Likelihood:                -174.76
No. Observations:                 154   AIC:                             513.5
Df Residuals:                      72   BIC:                             762.6
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------------
Plantains                           0.0016      0.002      0.998      0.322        -0.002     0.005
Sugar, Raw Equivalent               0.0121      0.009      1.290      0.201        -0.007     0.031
Beverages, Alcoholic                     0          0        nan        nan             0         0
Olives (including preserved)             0          0        nan        nan             0         0
Cloves                                   0          0        nan        nan             0         0
Coconuts - Incl Copra                    0          0        nan        nan             0         0
Vegetables, Other                   0.0005      0.001      0.316      0.753        -0.002     0.003
Sesame seed                              0          0        nan        nan             0         0
Wine                                     0          0        nan        nan             0         0
Apples and products                      0          0        nan        nan             0         0
Rape and Mustard Oil                     0          0        nan        nan             0         0
Maize and products                  0.0078      0.001      7.452      0.000         0.006     0.010
Groundnuts (Shelled Eq)                  0          0        nan        nan             0         0
Barley and products                      0          0        nan        nan             0         0
Maize Germ Oil                           0          0        nan        nan             0         0
Beer                                0.0011      0.001      0.792      0.431        -0.002     0.004
Groundnut Oil                            0          0        nan        nan             0         0
Pineapples and products                  0          0        nan        nan             0         0
Pulses, Other and products          0.0010      0.010      0.103      0.919        -0.018     0.020
Sugar (Raw Equivalent)              0.0002      0.010      0.022      0.983        -0.020     0.020
Palm Oil                                 0          0        nan        nan             0         0
Oilcrops, Other                          0          0        nan        nan             0         0
Dates                                    0          0        nan        nan             0         0
Oats                                     0          0        nan        nan             0         0
Soyabeans                                0          0        nan        nan             0         0
Beans                                    0          0        nan        nan             0         0
Sesameseed Oil                           0          0        nan        nan             0         0
Grapes and products (excl wine)          0          0        nan        nan        

Likewise, when considering the logarithmically transformed age-standardized mortality rate from cancer, the Omnibus and Jarque-Bera Tests reject the null hypothesis of normality in the residuals.

In [74]:
lasso = linear_model.Lasso(alpha = 1.0)
lasso.fit(food_1970_2000_cleaned, deaths_100k_cardio_2000_log)
y_deaths_100k_cardio_lasso = deaths_100k_cardio_2000_log.sort_index().loc[food_1970_2000_cleaned.index].values
lasso.score(food_1970_2000_cleaned, y_deaths_100k_cardio_lasso)
params = {
    'alpha': [0.3,0.6,1.0,1.3,1.6,2.0,2.3,2.6,3.0,4.0,5.0],
    'fit_intercept': [True, False],
    'normalize': [True, False],
}

CV_model = sk.grid_search.GridSearchCV(lasso, param_grid=params, cv=5)
CV_model.fit(food_1970_2000_cleaned, y_deaths_100k_cardio_lasso)
CV_model.best_params_

{'alpha': 0.6, 'fit_intercept': True, 'normalize': False}

In [75]:
deaths_cardio_model_lasso = sm.OLS(y_deaths_100k_cardio_lasso, food_1970_2000_cleaned)
deaths_cardio_results_lasso = deaths_cardio_model_lasso.fit_regularized(alpha = 0.6)
deaths_cardio_results_lasso.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.961
Method:                 Least Squares   F-statistic:                     47.47
Date:                Tue, 13 Dec 2016   Prob (F-statistic):           6.50e-42
Time:                        11:54:42   Log-Likelihood:                -178.04
No. Observations:                 154   AIC:                             520.1
Df Residuals:                      72   BIC:                             769.1
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------------
Plantains                           0.0031      0.002      1.801      0.076        -0.000     0.007
Sugar, Raw Equivalent               0.0107      0.004      2.697      0.009         0.003     0.019
Beverages, Alcoholic                     0          0        nan        nan             0         0
Olives (including preserved)             0          0        nan        nan             0         0
Cloves                                   0          0        nan        nan             0         0
Coconuts - Incl Copra                    0          0        nan        nan             0         0
Vegetables, Other               -7.326e-05      0.002     -0.039      0.969        -0.004     0.004
Sesame seed                        -0.0057      0.008     -0.754      0.453        -0.021     0.009
Wine                                     0          0        nan        nan             0         0
Apples and products                -0.0011      0.007     -0.152      0.880        -0.016     0.014
Rape and Mustard Oil               -0.0040      0.007     -0.540      0.591        -0.019     0.011
Maize and products                  0.0084      0.001      6.521      0.000         0.006     0.011
Groundnuts (Shelled Eq)             0.0014      0.015      0.090      0.929        -0.029     0.031
Barley and products                 0.0012      0.005      0.229      0.819        -0.009     0.012
Maize Germ Oil                           0          0        nan        nan             0         0
Beer                                0.0007      0.002      0.299      0.766        -0.004     0.005
Groundnut Oil                       0.0015      0.014      0.112      0.911        -0.026     0.029
Pineapples and products                  0          0        nan        nan             0         0
Pulses, Other and products          0.0365      0.011      3.412      0.001         0.015     0.058
Sugar (Raw Equivalent)                   0          0        nan        nan             0         0
Palm Oil                            0.0039      0.008      0.499      0.619        -0.012     0.019
Oilcrops, Other                     0.0024      0.007      0.357      0.722        -0.011     0.016
Dates                               0.0085      0.005      1.560      0.123        -0.002     0.019
Oats                                     0          0        nan        nan             0         0
Soyabeans                                0          0        nan        nan             0         0
Beans                               0.0069      0.009      0.761      0.449        -0.011     0.025
Sesameseed Oil                      0.0115      0.008      1.468      0.146        -0.004     0.027
Grapes and products (excl wine)     0.0010      0.005      0.191      0.849        

For the transformed age-standardized mortality rate from cardiovascular disease, once again the Omnibus and Jarque-Bera Tests reject the null hypothesis of normality in the residuals. Since the un-transformed model did appear to have normally distributed residuals, we should use that model for analysis instead.

In [67]:
lasso = linear_model.Lasso(alpha = 1.0)
lasso.fit(food_1970_2000_cleaned, deaths_100k_diabetes_2000_log)
y_deaths_100k_diabetes_lasso = deaths_100k_diabetes_2000_log.sort_index().loc[food_1970_2000_cleaned.index].values
lasso.score(food_1970_2000_cleaned, y_deaths_100k_diabetes_lasso)
params = {
    'alpha': [0.3,0.6,1.0,1.3,1.6,2.0,2.3,2.6,3.0,4.0,5.0],
    'fit_intercept': [True, False],
    'normalize': [True, False],
}

CV_model = sk.grid_search.GridSearchCV(lasso, param_grid=params, cv=5)
CV_model.fit(food_1970_2000_cleaned, y_deaths_100k_diabetes_lasso)
CV_model.best_params_

{'alpha': 1.0, 'fit_intercept': True, 'normalize': False}

In [69]:
deaths_diabetes_model_lasso = sm.OLS(y_deaths_100k_diabetes_lasso, food_1970_2000_cleaned)
deaths_diabetes_results_lasso = deaths_diabetes_model_lasso.fit_regularized(alpha = 1.0)
deaths_diabetes_results_lasso.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     20.19
Date:                Tue, 13 Dec 2016   Prob (F-statistic):           2.47e-29
Time:                        11:46:23   Log-Likelihood:                -157.57
No. Observations:                 154   AIC:                             479.1
Df Residuals:                      72   BIC:                             728.2
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------------
Plantains                           0.0018      0.001      1.236      0.221        -0.001     0.005
Sugar, Raw Equivalent               0.0079      0.008      0.941      0.350        -0.009     0.025
Beverages, Alcoholic                     0          0        nan        nan             0         0
Olives (including preserved)             0          0        nan        nan             0         0
Cloves                                   0          0        nan        nan             0         0
Coconuts - Incl Copra               0.0013      0.004      0.358      0.722        -0.006     0.008
Vegetables, Other                        0          0        nan        nan             0         0
Sesame seed                        -0.0011      0.005     -0.206      0.838        -0.012     0.010
Wine                                     0          0        nan        nan             0         0
Apples and products                -0.0022      0.005     -0.431      0.668        -0.012     0.008
Rape and Mustard Oil                     0          0        nan        nan             0         0
Maize and products                  0.0055      0.001      5.651      0.000         0.004     0.007
Groundnuts (Shelled Eq)                  0          0        nan        nan             0         0
Barley and products                 0.0029      0.004      0.646      0.520        -0.006     0.012
Maize Germ Oil                           0          0        nan        nan             0         0
Beer                                     0          0        nan        nan             0         0
Groundnut Oil                            0          0        nan        nan             0         0
Pineapples and products                  0          0        nan        nan             0         0
Pulses, Other and products          0.0246      0.009      2.807      0.006         0.007     0.042
Sugar (Raw Equivalent)              0.0038      0.009      0.427      0.670        -0.014     0.022
Palm Oil                                 0          0        nan        nan             0         0
Oilcrops, Other                          0          0        nan        nan             0         0
Dates                               0.0048      0.005      1.066      0.290        -0.004     0.014
Oats                                     0          0        nan        nan             0         0
Soyabeans                                0          0        nan        nan             0         0
Beans                                    0          0        nan        nan             0         0
Sesameseed Oil                           0          0        nan        nan             0         0
Grapes and products (excl wine)     0.0010      0.004      0.224      0.824        

While the Omnibus and Jarque-Bera Tests once again reject the null hypothesis of normality in the residuals for the transformed age-standardized mortality rate from diabetes, it is worth noting that the p values are larger than under the un-transformed model, which is a good sign.

In [70]:
lasso = linear_model.Lasso(alpha = 1.0)
lasso.fit(food_1970_2000_cleaned, deaths_100k_resp_2000_log)
y_deaths_100k_resp_lasso = deaths_100k_resp_2000_log.sort_index().loc[food_1970_2000_cleaned.index].values
lasso.score(food_1970_2000_cleaned, y_deaths_100k_resp_lasso)
params = {
    'alpha': [0.3,0.6,1.0,1.3,1.6,2.0,2.3,2.6,3.0,4.0,5.0],
    'fit_intercept': [True, False],
    'normalize': [True, False],
}

CV_model = sk.grid_search.GridSearchCV(lasso, param_grid=params, cv=5)
CV_model.fit(food_1970_2000_cleaned, y_deaths_100k_resp_lasso)
CV_model.best_params_

{'alpha': 1.0, 'fit_intercept': True, 'normalize': False}

In [71]:
deaths_resp_model_lasso = sm.OLS(y_deaths_100k_resp_lasso, food_1970_2000_cleaned)
deaths_resp_results_lasso = deaths_resp_model_lasso.fit_regularized(alpha = 1.0)
deaths_resp_results_lasso.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     26.14
Date:                Tue, 13 Dec 2016   Prob (F-statistic):           4.59e-33
Time:                        11:49:34   Log-Likelihood:                -157.82
No. Observations:                 154   AIC:                             479.6
Df Residuals:                      72   BIC:                             728.7
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------------
Plantains                           0.0017      0.001      1.214      0.229        -0.001     0.005
Sugar, Raw Equivalent               0.0047      0.003      1.770      0.081        -0.001     0.010
Beverages, Alcoholic                     0          0        nan        nan             0         0
Olives (including preserved)             0          0        nan        nan             0         0
Cloves                                   0          0        nan        nan             0         0
Coconuts - Incl Copra                    0          0        nan        nan             0         0
Vegetables, Other                   0.0001      0.001      0.094      0.926        -0.003     0.003
Sesame seed                              0          0        nan        nan             0         0
Wine                               -0.0002      0.003     -0.061      0.951        -0.005     0.005
Apples and products                      0          0        nan        nan             0         0
Rape and Mustard Oil               -0.0039      0.006     -0.635      0.527        -0.016     0.008
Maize and products                  0.0070      0.001      6.873      0.000         0.005     0.009
Groundnuts (Shelled Eq)                  0          0        nan        nan             0         0
Barley and products                      0          0        nan        nan             0         0
Maize Germ Oil                           0          0        nan        nan             0         0
Beer                                     0          0        nan        nan             0         0
Groundnut Oil                            0          0        nan        nan             0         0
Pineapples and products                  0          0        nan        nan             0         0
Pulses, Other and products          0.0268      0.009      3.077      0.003         0.009     0.044
Sugar (Raw Equivalent)                   0          0        nan        nan             0         0
Palm Oil                                 0          0        nan        nan             0         0
Oilcrops, Other                          0          0        nan        nan             0         0
Dates                                    0          0        nan        nan             0         0
Oats                                     0          0        nan        nan             0         0
Soyabeans                                0          0        nan        nan             0         0
Beans                               0.0004      0.008      0.052      0.959        -0.015     0.015
Sesameseed Oil                      0.0009      0.006      0.145      0.885        -0.011     0.013
Grapes and products (excl wine)          0          0        nan        nan        

With the transformed age-standardized mortality rate from respiratory disease as the response variable, the Omnibus and Jarque-Bera Tests still reject the null hypothesis of normality in the residuals.